<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/RAG/Knowledge_Graph/Extract_themes_Microsoft_Phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phi3
https://github.com/microsoft/Phi-3CookBook

In [8]:
!python -m pip install langchain langchain-google-genai langchain langchain-community langchain-experimental --quiet
! python -m pip install graphdatascience nltk  accelerate flash_attn einops timm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.9 MB/s eta 0:00:00


In [12]:
! pip install -U accelerate transformers  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 64.5 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from graphdatascience import GraphDataScience
from getpass import getpass
import time
import numpy as np
import nltk
from google.oauth2 import service_account

import json

import os

# Set up connection information

In [3]:
neo4j_password = "topic-birth-minute"

In [4]:
neo4j_database_name = "neo4j"
neo4j_user = "neo4j"
bolt_url = "bolt://34.205.131.10:7687"
gds = GraphDataScience(bolt_url, auth=(neo4j_user, neo4j_password))

I was curious to experiment with the Claude 3 models for this, but other models should work fine.

In [12]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",

    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

messages = [
    {"role": "system", "content": "Your are a python developer."},
    {"role": "user", "content": "Help me generate a bubble algorithm"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 600,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Sure, I can help you with that. The bubble sort algorithm is a simple sorting algorithm that repeatedly steps through the list, compares adjacent elements and swaps them if they are in the wrong order. The pass through the list is repeated until the list is sorted.

Here's a Python implementation of the bubble sort algorithm:

```python
def bubble_sort(arr):
    n = len(arr)
    # Traverse through all array elements
    for i in range(n):
        # Last i elements are already in place
        for j in range(0, n-i-1):
            # Traverse the array from 0 to n-i-1
            # Swap if the element found is greater than the next element
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]

# Example usage
arr = [64, 34, 25, 12, 22, 11, 90]
bubble_sort(arr)
print("Sorted array is:", arr)
```

In this example, we define a function `bubble_sort` that takes an array as input and sorts it using the bubble sort algorithm. The outer loop runs `n` times, wher

In [84]:
from langchain.llms.base import LLM

class CustomLLM(LLM):
    def _call(self, prompt, stop=None, run_manager=None) -> str:
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = inputs.to("cuda")
        result = model.generate(input_ids=inputs.input_ids, max_new_tokens=1024, temperature=0.2, do_sample=True)
        #
        prompt_length = len(tokenizer.encode(prompt, return_tensors="pt")[0])
        generated_response = tokenizer.decode(
            result[0][prompt_length:], skip_special_tokens=True
        )
        # result = tokenizer.decode(result[0])
        return generated_response

    @property
    def _llm_type(self) -> str:
        return "custom"

llm = CustomLLM()

In [85]:
llm

CustomLLM()

# Get movie data

In [106]:
movie_info = gds.run_cypher(
    """MATCH (m:Movie)
       WHERE NOT EXISTS {(m)-[:HAS_THEME]->()}
       AND trim(m.overview) <> ''
       RETURN m.tmdbId AS tmdbId, m{.title, .overview} AS description""" )

In [107]:
movie_info.head()

,tmdbId,description


In [62]:
len(movie_info)

23742

In [63]:
movie_info[:1].to_dict()

{'tmdbId': {0: 42050},
 'description': {0: {'title': 'Seduction: The Cruel Woman',
   'overview': 'Wanda is a dominatrix who runs a gallery in a building on the Hamburg waterfront, where audiences pay for the privilege of watching her humiliate her slaves. She is a business woman who smashes sexual stereotypes and social taboos with icy self-possession and an enigmatic smile. As artist she specializes in the staging of elaborate BDSM fantasies and her affairs transgress the usual boundaries of personal and professional life. Along the way she leaves her German lesbian lover, a shoe fetishist, for an American "trainee," and does more than step on the toes of the male performer who has broken the rules of the master-slave relationship by falling in love with her.'}}}

# Create theme extraction pipeline

In [86]:
prompt_1 ="""You are a movie expert.
    You are given the tile and overview of the plot of a movie.
    Summarize the most memorable themes, settings, and public figures in the movie into a list of up to eight one-to-two word phrases.
    Only include the names of people if the person is a famous public figure.
    Prioritize any phrases that appear in the movie's title.
    You can provide fewer than eight phrases.
    very important:
    1) Return the phrases as a pipe separated list.
    2) Return only the list without a heading.
    title: {title}
    overview: {overview}
    """
chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

template = f'{chat_template.format(input=prompt_1)}'

final_prompt = ChatPromptTemplate.from_template(template)



In [87]:
final_prompt

ChatPromptTemplate(input_variables=['overview', 'title'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['overview', 'title'], template="<|user|>\nYou are a movie expert. \n    You are given the tile and overview of the plot of a movie.\n    Summarize the most memorable themes, settings, and public figures in the movie into a list of up to eight one-to-two word phrases. \n    Only include the names of people if the person is a famous public figure.\n    Prioritize any phrases that appear in the movie's title.\n    You can provide fewer than eight phrases.\n    very important:\n    1) Return the phrases as a pipe separated list. \n    2) Return only the list without a heading.\n    title: {title}\n    overview: {overview} \n     <|end|>\n<|assistant|>"))])

In [88]:
summary_chain = final_prompt | llm

In [89]:
movie_info.loc[1, 'description']

{'title': 'The Atrocity Exhibition',
 'overview': 'A doctor in a mental research institution is driven insane by the spectacle of the horrors of the twentieth century.'}

In [90]:
out = summary_chain.invoke(movie_info.loc[1, 'description'])

In [91]:
out

'Atrocity Exhibition | Doctor | Mental Research | Twentieth Century | Insanity | Spectacle | Horrors | Public Figures'

In [92]:
test_batch = movie_info.loc[0:5,:]['description'].to_list()

In [93]:
summary_chain.batch(test_batch)

['Seduction|Cruel Woman|Wanda|Dominatrix|Gallery|Hamburg|BDSM|Sexual Stereotypes|Artist|Lesbian|Shoe Fetishist|American|Trainee|Master-Slave|Performer|Toes',
 'Atrocity Exhibition | Doctor | Mental Research Institution | Twentieth Century | Insanity | Horrors | Spectacle | Public Figure',
 'Legend | DJ | Serious | Alan Walker | Illenium | Dj Khaled | Beat Matching | Nightclub | Studio',
 '90s Rave Culture | Diverse Characters | Self-Discovery | Underground Parties | Nostalgic Journey | Drugs | Dance | 1990s | Universal Groove',
 'Circus, Love, Boy, Leave, Life, Love Song, Public Figure, Famous',
 'Beverly Hills | Axel Foley | Conspiracy | Detective | Wisecracking | Partner | Old Pals | Threat']

# Extract themes and send to Neo4j
Create a node key constraint for the Theme nodes that requires that all Theme nodes have a unique value for the description property.

In [94]:
gds.run_cypher("""CREATE CONSTRAINT theme_node_key IF NOT EXISTS FOR (n:Theme) REQUIRE n.description IS NODE KEY""")

""


In [104]:
def process_theme_batch(start_index, end_index):
    rows = movie_info.iloc[start_index: end_index,:].copy()
    results = summary_chain.batch(rows['description'].to_list())
    # r= [result.content.split("|") for result in results]
    themes =[]
    for result in results:
        if result != "":
            themes.append(result.replace(",", "|").split("|"))
        else:
            themes.append([])
    rows['themes'] = themes
    gds.run_cypher("""
    UNWIND $data as d
    MATCH (m:Movie {tmdbId:d['tmdbId']})
    UNWIND d['themes'] AS theme
    MERGE (t:Theme {description:trim(theme)})
    WITH t, m
    MERGE (m)-[:HAS_THEME {prompt: "v2"}]->(t)
    """,
                   {"data": rows[['tmdbId', 'themes']].to_dict("records")})

In [105]:
for i in range(0, int(movie_info.shape[0]/5) + 1):
    process_theme_batch(i*5,(i+1)*5)
    if (i + 1) % 5 == 0:
        print(f"Finished row {(i+1)*5}")
    time.sleep(1)

Finished row 25
Finished row 50
Finished row 75
Finished row 100
Finished row 125
Finished row 150
Finished row 175
Finished row 200
Finished row 225
Finished row 250
Finished row 275
Finished row 300
Finished row 325
Finished row 350
Finished row 375
Finished row 400
Finished row 425
Finished row 450
Finished row 475
Finished row 500
Finished row 525
Finished row 550
Finished row 575
Finished row 600
Finished row 625
Finished row 650
Finished row 675
Finished row 700
Finished row 725
Finished row 750
Finished row 775
Finished row 800
Finished row 825
Finished row 850
Finished row 875
Finished row 900
Finished row 925
Finished row 950
Finished row 975
Finished row 1000
Finished row 1025
Finished row 1050
Finished row 1075
Finished row 1100
Finished row 1125
Finished row 1150
Finished row 1175
Finished row 1200
Finished row 1225
Finished row 1250
Finished row 1275
Finished row 1300
Finished row 1325
Finished row 1350
Finished row 1375
Finished row 1400
Finished row 1425
Finished row 145